In [513]:
# Importation des bibliothèques
import numpy as np
import pandas as pd

# Importation de l'API
from nba_api.stats.endpoints import leaguegamefinder

In [514]:
# Récupération du jeu de données
path = 'nba odds 2021-22.xlsx'
odds_df = pd.read_excel(path, usecols = ['Date', 'Team', 'Final', 'ML'])
odds_df.tail()

,Date,Team,Final,ML
2641,610,Boston,97,-165
2642,613,Boston,94,+145
2643,613,Golden State,104,-165
2644,616,Golden State,103,+155
2645,616,Boston,90,-175


In [515]:
# Suppression des espaces dans les noms des équipes
odds_df.Team = odds_df.Team.str.replace(' ', '')
odds_df.tail()

,Date,Team,Final,ML
2641,610,Boston,97,-165
2642,613,Boston,94,+145
2643,613,GoldenState,104,-165
2644,616,GoldenState,103,+155
2645,616,Boston,90,-175


In [516]:
# Création d'un dictionnaire pour remplacer les équipes par leurs noms officiels
team_name = {'Detroit': 'DetroitPistons', 'Washington': 'WashingtonWizards',
            'Dallas': 'DallasMavericks', 'Phoenix': 'PhoenixSuns',
            'NewOrleans': 'NewOrleansPelicans', 'LAClippers': 'LAClippers',
            'OklahomaCity': 'OklahomaCityThunder', 'GoldenState': 'GoldenStateWarriors',
            'Philadelphia': 'Philadelphia76ers', 'Indiana': 'IndianaPacers',
            'Miami': 'MiamiHeat', 'Toronto': 'TorontoRaptors',
            'Orlando': 'OrlandoMagic', 'NewYork': 'NewYorkKnicks',
            'Boston': 'BostonCeltics', 'Chicago': 'ChicagoBulls',
            'SanAntonio': 'SanAntonioSpurs', 'Portland': 'PortlandTrailBlazers',
            'Denver': 'DenverNuggets', 'Memphis': 'MemphisGrizzlies',
            'Brooklyn': 'BrooklynNets', 'Houston': 'HoustonRockets',
            'Utah': 'UtahJazz', 'Minnesota':'MinnesotaTimberwolves',
            'LALakers': 'LosAngelesLakers', 'Atlanta': 'AtlantaHawks',
            'Charlotte': 'CharlotteHornets', 'Cleveland': 'ClevelandCavaliers',
            'Sacramento': 'SacramentoKings', 'Milwaukee': 'MilwaukeeBucks'}
odds_df.replace({'Team': team_name}, inplace = True)

In [517]:
def format(date):
    'Fonction pour changer le format de l\'heure de notre dataframe'
    'Elle prend notre dataframe en entrée'
    'Elle nous retourne une date qui respecte le format AAAAMMJJ'
    
    year = path.split(' ')[2].split('.')[0].split('-')[0]
    
    if odds_df['Date'][0] <= date:
        return year + str(date)
    else:
        return str(int(year) + 1) + '0' + str(date)

In [518]:
# Formattage des données pour la création de notre identifiant
odds_df['Date'] = list(map(format, odds_df['Date']))
odds_df['ID'] = list(map(str, odds_df['Date'])) + odds_df['Team']
odds_df.tail()

,Date,Team,Final,ML,ID
2641,20220610,BostonCeltics,97,-165,20220610BostonCeltics
2642,20220613,BostonCeltics,94,+145,20220613BostonCeltics
2643,20220613,GoldenStateWarriors,104,-165,20220613GoldenStateWarriors
2644,20220616,GoldenStateWarriors,103,+155,20220616GoldenStateWarriors
2645,20220616,BostonCeltics,90,-175,20220616BostonCeltics


In [519]:
def change_odds(x):
    'Fonction pour changer le format des cotes de notre dataframe'
    'Elle prend une cote américaine en entrée'
    'Elle nous retourne une cote qui correspond à la cote française'

    return round(1 + x / 100 , 2) if x > 0 else round(1 + 100 / np.abs(x),2)

In [520]:
# Modification du format des cotes
odds_df['ML'] = odds_df['ML'].astype(int)
odds_df['ODDS'] = odds_df['ML'].apply(lambda x : change_odds(x))
odds_df.head()

,Date,Team,Final,ML,ID,ODDS
0,20211019,BrooklynNets,104,105,20211019BrooklynNets,2.05
1,20211019,MilwaukeeBucks,127,-125,20211019MilwaukeeBucks,1.80
2,20211019,GoldenStateWarriors,121,140,20211019GoldenStateWarriors,2.40
3,20211019,LosAngelesLakers,114,-160,20211019LosAngelesLakers,1.62
4,20211020,IndianaPacers,122,-125,20211020IndianaPacers,1.80


In [521]:
# Récupération du deuxième jeu de données
score_df = leaguegamefinder.LeagueGameFinder().get_data_frames()[0]
score_df.drop(['SEASON_ID', 'TEAM_ABBREVIATION', 'MIN'], axis = 1, inplace = True)
score_df.head()

,TEAM_ID,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,FGM,FGA,FG_PCT,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,1610612757,Portland Trail Blazers,0022200616,2023-01-10,POR vs. ORL,L,106,42,87,0.483,...,0.842,13,30,43,22,10,7,16,22,-3.0
1,1610612762,Utah Jazz,0022200614,2023-01-10,UTA vs. CLE,W,116,38,87,0.437,...,0.811,15,39,54,21,3,3,15,17,2.0
2,1612709930,G League Ignite,2022200091,2023-01-10,GLI vs. MXC,W,123,41,77,0.532,...,0.952,9,25,34,16,10,3,16,27,1.0
3,1610612746,LA Clippers,0022200617,2023-01-10,LAC vs. DAL,W,113,39,86,0.453,...,0.767,12,34,46,20,7,4,9,23,12.0
4,1612709903,Salt Lake City Stars,2022200090,2023-01-10,SLC vs. RGV,L,122,47,95,0.495,...,0.500,16,34,50,28,3,7,18,18,-10.2


In [522]:
# Suppression des tirets dans les dates et des espaces dans les noms des équipes
score_df.GAME_DATE = score_df.GAME_DATE.str.replace('-', '')
score_df.TEAM_NAME = score_df.TEAM_NAME.str.replace(' ', '')
score_df.head()

,TEAM_ID,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,FGM,FGA,FG_PCT,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,1610612757,PortlandTrailBlazers,0022200616,20230110,POR vs. ORL,L,106,42,87,0.483,...,0.842,13,30,43,22,10,7,16,22,-3.0
1,1610612762,UtahJazz,0022200614,20230110,UTA vs. CLE,W,116,38,87,0.437,...,0.811,15,39,54,21,3,3,15,17,2.0
2,1612709930,GLeagueIgnite,2022200091,20230110,GLI vs. MXC,W,123,41,77,0.532,...,0.952,9,25,34,16,10,3,16,27,1.0
3,1610612746,LAClippers,0022200617,20230110,LAC vs. DAL,W,113,39,86,0.453,...,0.767,12,34,46,20,7,4,9,23,12.0
4,1612709903,SaltLakeCityStars,2022200090,20230110,SLC vs. RGV,L,122,47,95,0.495,...,0.500,16,34,50,28,3,7,18,18,-10.2


In [523]:
# Suppression de la colonne MATCHUP et création des colonnes HOME et AWAY
score_df['HOME'] = [1 if '@' in score_df.MATCHUP[index] else 0 for index, row in score_df.iterrows()]
score_df['AWAY'] = [1 if 'vs.' in score_df.MATCHUP[index] else 0 for index, row in score_df.iterrows()]
score_df.drop(['MATCHUP'], axis = 1, inplace = True)
score_df.head()

,TEAM_ID,TEAM_NAME,GAME_ID,GAME_DATE,WL,PTS,FGM,FGA,FG_PCT,FG3M,...,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HOME,AWAY
0,1610612757,PortlandTrailBlazers,0022200616,20230110,L,106,42,87,0.483,6,...,30,43,22,10,7,16,22,-3.0,0,1
1,1610612762,UtahJazz,0022200614,20230110,W,116,38,87,0.437,10,...,39,54,21,3,3,15,17,2.0,0,1
2,1612709930,GLeagueIgnite,2022200091,20230110,W,123,41,77,0.532,8,...,25,34,16,10,3,16,27,1.0,0,1
3,1610612746,LAClippers,0022200617,20230110,W,113,39,86,0.453,12,...,34,46,20,7,4,9,23,12.0,0,1
4,1612709903,SaltLakeCityStars,2022200090,20230110,L,122,47,95,0.495,16,...,34,50,28,3,7,18,18,-10.2,0,1


In [524]:
# Suppression de la colonne WL et création des colonnes W et L
score_df = pd.get_dummies(score_df , prefix = '', prefix_sep='', columns=['WL'])
score_df.head()

,TEAM_ID,TEAM_NAME,GAME_ID,GAME_DATE,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,...,AST,STL,BLK,TOV,PF,PLUS_MINUS,HOME,AWAY,L,W
0,1610612757,PortlandTrailBlazers,0022200616,20230110,106,42,87,0.483,6,35,...,22,10,7,16,22,-3.0,0,1,1,0
1,1610612762,UtahJazz,0022200614,20230110,116,38,87,0.437,10,36,...,21,3,3,15,17,2.0,0,1,0,1
2,1612709930,GLeagueIgnite,2022200091,20230110,123,41,77,0.532,8,23,...,16,10,3,16,27,1.0,0,1,0,1
3,1610612746,LAClippers,0022200617,20230110,113,39,86,0.453,12,31,...,20,7,4,9,23,12.0,0,1,0,1
4,1612709903,SaltLakeCityStars,2022200090,20230110,122,47,95,0.495,16,38,...,28,3,7,18,18,-10.2,0,1,1,0


In [525]:
# On supprime toutes les lignes contenant une équipe qui ne fait pas partie des 30 équipes de la NBA 
teams = ['AtlantaHawks','BrooklynNets','BostonCeltics', 'CharlotteHornets', 'ChicagoBulls', 'ClevelandCavaliers', 
'DallasMavericks', 'DenverNuggets', 'DetroitPistons', 'GoldenStateWarriors', 'HoustonRockets', 'IndianaPacers', 
'LAClippers', 'LosAngelesLakers', 'MemphisGrizzlies', 'MiamiHeat', 'MilwaukeeBucks', 'MinnesotaTimberwolves',
'NewOrleansPelicans', 'NewYorkKnicks', 'OklahomaCityThunder', 'OrlandoMagic', 'Philadelphia76ers', 'PhoenixSuns', 
'PortlandTrailBlazers', 'SacramentoKings', 'SanAntonioSpurs', 'TorontoRaptors', 'UtahJazz', 'WashingtonWizards']

team_serie = pd.Series(score_df["TEAM_NAME"])
score_df = score_df[team_serie.str.contains('|'.join(teams))]

In [526]:
# On supprime toutes les lignes qui possède un GAME_ID unique
game_id_counts = score_df["GAME_ID"].value_counts()
single_occurrence_game_ids = game_id_counts[game_id_counts == 1].index

score_df = score_df[~score_df["GAME_ID"].isin(single_occurrence_game_ids)]
score_df.head()

,TEAM_ID,TEAM_NAME,GAME_ID,GAME_DATE,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,...,AST,STL,BLK,TOV,PF,PLUS_MINUS,HOME,AWAY,L,W
0,1610612757,PortlandTrailBlazers,0022200616,20230110,106,42,87,0.483,6,35,...,22,10,7,16,22,-3.0,0,1,1,0
1,1610612762,UtahJazz,0022200614,20230110,116,38,87,0.437,10,36,...,21,3,3,15,17,2.0,0,1,0,1
3,1610612746,LAClippers,0022200617,20230110,113,39,86,0.453,12,31,...,20,7,4,9,23,12.0,0,1,0,1
5,1610612742,DallasMavericks,0022200617,20230110,101,30,69,0.435,12,38,...,15,5,6,11,17,-12.0,1,0,1,0
6,1610612761,TorontoRaptors,0022200613,20230110,132,45,91,0.495,20,44,...,32,4,4,11,17,12.0,0,1,0,1


In [527]:
# Création d'un indice d'efficaicité lors des matchs et suppression des colonnes inutiles
score_df['EFF'] = (score_df['PTS'] + score_df['REB'] + score_df['AST'] + score_df['STL'] + score_df['BLK'] - ((score_df['FGA'] + score_df['FGM'])+(score_df['FTA']-score_df['FTM'])+score_df['TOV']))
score_df.drop(['PTS', 'REB', 'AST', 'STL', 'BLK', 'FGA', 'FGM', 'FTA', 'FTM', 'TOV', 'FG3M', 'FG3A', 'OREB', 'DREB'], axis = 1, inplace = True)
score_df.head()

,TEAM_ID,TEAM_NAME,GAME_ID,GAME_DATE,FG_PCT,FG3_PCT,FT_PCT,PF,PLUS_MINUS,HOME,AWAY,L,W,EFF
0,1610612757,PortlandTrailBlazers,0022200616,20230110,0.483,0.171,0.842,22,-3.0,0,1,1,0,40
1,1610612762,UtahJazz,0022200614,20230110,0.437,0.278,0.811,17,2.0,0,1,0,1,50
3,1610612746,LAClippers,0022200617,20230110,0.453,0.387,0.767,23,12.0,0,1,0,1,49
5,1610612742,DallasMavericks,0022200617,20230110,0.435,0.316,0.806,17,-12.0,1,0,1,0,42
6,1610612761,TorontoRaptors,0022200613,20230110,0.495,0.455,0.815,17,12.0,0,1,0,1,69


In [528]:
# Création de notre identifiant pour le merge de nos deux dataframes
score_df['ID'] = score_df['GAME_DATE'] + score_df['TEAM_NAME']
score_df.head()

,TEAM_ID,TEAM_NAME,GAME_ID,GAME_DATE,FG_PCT,FG3_PCT,FT_PCT,PF,PLUS_MINUS,HOME,AWAY,L,W,EFF,ID
0,1610612757,PortlandTrailBlazers,0022200616,20230110,0.483,0.171,0.842,22,-3.0,0,1,1,0,40,20230110PortlandTrailBlazers
1,1610612762,UtahJazz,0022200614,20230110,0.437,0.278,0.811,17,2.0,0,1,0,1,50,20230110UtahJazz
3,1610612746,LAClippers,0022200617,20230110,0.453,0.387,0.767,23,12.0,0,1,0,1,49,20230110LAClippers
5,1610612742,DallasMavericks,0022200617,20230110,0.435,0.316,0.806,17,-12.0,1,0,1,0,42,20230110DallasMavericks
6,1610612761,TorontoRaptors,0022200613,20230110,0.495,0.455,0.815,17,12.0,0,1,0,1,69,20230110TorontoRaptors


In [529]:
# Merge des deux dataframes
df_merge = score_df.merge(odds_df, left_on = 'ID', right_on='ID')
df_merge.head()

,TEAM_ID,TEAM_NAME,GAME_ID,GAME_DATE,FG_PCT,FG3_PCT,FT_PCT,PF,PLUS_MINUS,HOME,AWAY,L,W,EFF,ID,Date,Team,Final,ML,ODDS
0,1610612738,BostonCeltics,0042100406,20220616,0.425,0.393,0.917,16,-13.0,0,1,1,0,37,20220616BostonCeltics,20220616,BostonCeltics,90,-175,1.57
1,1610612744,GoldenStateWarriors,0042100406,20220616,0.413,0.413,1.000,20,13.0,1,0,0,1,49,20220616GoldenStateWarriors,20220616,GoldenStateWarriors,103,155,2.55
2,1610612738,BostonCeltics,0042100405,20220613,0.413,0.344,0.677,16,-10.0,1,0,1,0,29,20220613BostonCeltics,20220613,BostonCeltics,94,145,2.45
3,1610612744,GoldenStateWarriors,0042100405,20220613,0.466,0.225,0.867,28,10.0,0,1,0,1,40,20220613GoldenStateWarriors,20220613,GoldenStateWarriors,104,-165,1.61
4,1610612738,BostonCeltics,0042100404,20220610,0.400,0.395,0.737,17,-10.0,0,1,1,0,36,20220610BostonCeltics,20220610,BostonCeltics,97,-165,1.61


In [530]:
# Suppression des dernières colonnes inutiles
df_merge.drop(['ID', 'Date', 'Team', 'ML'], axis = 1, inplace = True)
df_merge.head()

,TEAM_ID,TEAM_NAME,GAME_ID,GAME_DATE,FG_PCT,FG3_PCT,FT_PCT,PF,PLUS_MINUS,HOME,AWAY,L,W,EFF,Final,ODDS
0,1610612738,BostonCeltics,0042100406,20220616,0.425,0.393,0.917,16,-13.0,0,1,1,0,37,90,1.57
1,1610612744,GoldenStateWarriors,0042100406,20220616,0.413,0.413,1.000,20,13.0,1,0,0,1,49,103,2.55
2,1610612738,BostonCeltics,0042100405,20220613,0.413,0.344,0.677,16,-10.0,1,0,1,0,29,94,2.45
3,1610612744,GoldenStateWarriors,0042100405,20220613,0.466,0.225,0.867,28,10.0,0,1,0,1,40,104,1.61
4,1610612738,BostonCeltics,0042100404,20220610,0.400,0.395,0.737,17,-10.0,0,1,1,0,36,97,1.61
